In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.colab import drive
drive.mount('/content/drive')
dir_path = "/content/drive/Shareddrives/Rays-Proyecto Integrador/Python-Firebase/bitesight-858b3-firebase-adminsdk-rikzy-3c4a947c87.json"
cred = credentials.Certificate(dir_path)
firebase_admin.initialize_app(cred)

db = firestore.client()

Mounted at /content/drive


In [3]:
from collections import defaultdict
from scipy.spatial.distance import cosine

In [28]:
# Obtiene todos los datos de usuarios y ratings

users_docs = db.collection('users').stream()
users_data = {}
for doc in users_docs:
    user_id = doc.id
    user_data = doc.to_dict()
    users_data[user_id] = user_data

ratings_docs = db.collection('ratings').stream()
ratings_data = defaultdict(dict)
for doc in ratings_docs:
    data = doc.to_dict()
    user_id = data['user_id']
    restaurant_id = data['restaurant_id']
    rating = data['rating']
    ratings_data[user_id][restaurant_id] = rating

In [15]:
def get_user_preferences(user_id):
    user_doc = db.collection('users').document(user_id).get()
    if user_doc.exists:
        users_data[user_id] = user_doc.to_dict()
    else:
        raise ValueError(f'User with id {user_id} not found in the database')

    user_data = users_data[user_id]
    print(user_data['likes'])
    cuisine_preferences = user_data['likes']
    # pricing_preference = user_data['pricing']

    return cuisine_preferences
    # return cuisine_preferences, pricing_preference

def get_user_ratings(user_id):
    user_ratings = ratings_data[user_id]
    
    return user_ratings

def get_similar_users(user_id):
    # target_user_preferences, target_user_pricing = get_user_preferences(user_id)
    target_user_preferences = get_user_preferences(user_id)
    target_user_ratings = get_user_ratings(user_id)
    
    similarity_scores = []
    
    # Obtiene similitud de cosenos para todos los usuarios
    for other_user_id, other_user_data in users_data.items():
        if other_user_id == user_id:
            continue
        if 'likes' not in users_data[other_user_id]:
            continue
        
        # other_user_preferences, other_user_pricing = get_user_preferences(other_user_id)
        other_user_preferences = get_user_preferences(other_user_id)
        other_user_ratings = get_user_ratings(other_user_id)
        
        # Calcula la similitud basada en las preferencias de comida
        similarity_score = 1 - cosine(list(target_user_preferences.values()), list(other_user_preferences.values()))
        # Agrega a esta similitud las preferencias de precio
        # similarity_score += 1 - abs(target_user_pricing - other_user_pricing) / 4
        similarity_scores.append((similarity_score, other_user_id))
    
    similarity_scores.sort(reverse=True)
    similar_users = [user_id for _, user_id in similarity_scores[:10]]
    
    return similar_users

def get_restaurant_recommendations(user_id):
    similar_users = get_similar_users(user_id)
    restaurant_scores = defaultdict(int)
    
    for other_user_id in similar_users:
        other_user_ratings = get_user_ratings(other_user_id)
        # Suma o resta puntos dependiendo del rating de usuarios similares a cada restaurante
        # Solamente recomienda restaurantes a los que otros han dado un rating
        for restaurant_id, rating in other_user_ratings.items():
            restaurant_scores[restaurant_id] += (rating - 3)
    
    sorted_restaurant_scores = sorted(restaurant_scores.items(), key=lambda x: x[1], reverse=True)
    recommended_restaurants = [restaurant_id for restaurant_id, _ in sorted_restaurant_scores[:10]]
    
    return recommended_restaurants

In [32]:
user_id = '6s8K5XJ11KVmKnJJzCAL'
recommended_restaurants = get_restaurant_recommendations(user_id)

In [33]:
for restaurant_id in recommended_restaurants:
  restaurant_doc = db.collection('restaurants').document(restaurant_id).get()
  restaurant_data = restaurant_doc.to_dict()
  print(restaurant_data['title'])

LA FONDITA
EL ARCO RESTAURANTE
Fastory Restaurant
RESTAURANT CHENTA
Cuitoles Cocina - Huasteca
La Herencia Restaurante
Toks Mitras
Asador Las Diligencias
La Puntada Restaurante
Las Pampas Restaurante


In [25]:
users_data['0y0bUgw54pHiWIF8P5aq']

{'birthday': '1984/03/23',
 'likes': {'Healthy': True,
  'American': False,
  'Wings': True,
  'Italian': True,
  'Seafood': False,
  'Steakhouse': True,
  'Delicatessen': True,
  'Asian': True,
  'Mexican': True,
  'Ethnic': True,
  'Takeout': True,
  'Brewery': True},
 'email': 'thomas.brown@example.com',
 'gender': 'male',
 'name': 'Thomas Brown',
 'pricing': 3}

In [26]:
users_data['6s8K5XJ11KVmKnJJzCAL']

{'name': 'Gustavo Adolfo Benítez Leonés',
 'likes': {'Seafood': True,
  'Ethnic': False,
  'Wings': False,
  'Healthy': True,
  'Brewery': True,
  'American': True,
  'Italian': False,
  'Asian': False,
  'Takeout': False,
  'Steakhouse': False,
  'Mexican': True,
  'Delicatessen': False},
 'email': 'a00826760@tec.mx'}